In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from openalgo import api
from math import sqrt

# =========================
# CONFIG
# =========================
SYMBOL = "IRCON"
EXCHANGE = "NSE"
INTERVAL = "15m"       
START_DATE = "2025-11-01"
END_DATE = "2025-12-31"

INITIAL_CAPITAL = 1_000_000
RISK_PER_TRADE = 0.02   # 2%
STOP_LOSS_PCT = 0.01    # 1%
TAKE_PROFIT_PCT = 0.028 # 2.8%

# =========================
# CONNECT
# =========================
client = api(
    api_key="945cf5ba60f139afd48ce073fef0c346a2cc52d72f0c10c58e30acb50e0cb030",
    host="http://127.0.0.1:5000"
)

# =========================
# FETCH DATA
# =========================
df = client.history(
    symbol=SYMBOL,
    exchange=EXCHANGE,
    interval=INTERVAL,
    start_date=START_DATE,
    end_date=END_DATE
)

df.index = pd.to_datetime(df.index)

# =========================
# MODIFIED TREND BAR MOMENTUM STRATEGY
# =========================
df["prev_high"] = df["high"].shift(1)
df["prev_low"] = df["low"].shift(1)

df["signal"] = 0
df.loc[df["close"] > df["prev_high"], "signal"] = 1     # Buy
df.loc[df["close"] < df["prev_low"], "signal"] = -1     # Short

# =========================
# BACKTEST ENGINE
# =========================
capital = INITIAL_CAPITAL
position = 0
entry_price = 0
trade_log = []

equity_curve = []

for i in range(1, len(df)):
    row = df.iloc[i]
    price = row["close"]
    time = df.index[i]

    equity_curve.append(capital + position * (price - entry_price))

    # Exit logic
    if position != 0:
        if position == 1:
            sl = entry_price * (1 - STOP_LOSS_PCT)
            tp = entry_price * (1 + TAKE_PROFIT_PCT)
            exit_cond = price <= sl or price >= tp or row["signal"] == -1
        else:
            sl = entry_price * (1 + STOP_LOSS_PCT)
            tp = entry_price * (1 - TAKE_PROFIT_PCT)
            exit_cond = price >= sl or price <= tp or row["signal"] == 1

        if exit_cond:
            pnl = position * (price - entry_price)
            capital += pnl

            trade_log.append([entry_time, time, position, entry_price, price, pnl])

            position = 0

    # Entry logic
    if position == 0:
        risk_amount = capital * RISK_PER_TRADE
        qty = risk_amount / (price * STOP_LOSS_PCT)

        if row["signal"] == 1:
            position = qty
            entry_price = price
            entry_time = time

        elif row["signal"] == -1:
            position = -qty
            entry_price = price
            entry_time = time

# =========================
# TRADE LOG CSV
# =========================
trades_df = pd.DataFrame(trade_log, columns=[
    "Entry Time", "Exit Time", "Position", "Entry Price", "Exit Price", "PnL"
])

csv_path = "modified_trend_bar_momentum_trades.csv"
trades_df.to_csv(csv_path, index=False)

# =========================
# PERFORMANCE METRICS
# =========================
equity_series = pd.Series(equity_curve).dropna()
returns = equity_series.pct_change().dropna()

total_return = (capital - INITIAL_CAPITAL) / INITIAL_CAPITAL
years = len(df) / 252
cagr = (capital / INITIAL_CAPITAL) ** (1 / years) - 1 if years > 0 else 0

sharpe = sqrt(252) * returns.mean() / returns.std() if returns.std() != 0 else 0

drawdown = equity_series / equity_series.cummax() - 1
max_dd = drawdown.min()

win_rate = (trades_df["PnL"] > 0).mean() * 100 if len(trades_df) else 0

print("\n========== PERFORMANCE ==========")
print(f"Final Capital: ₹{capital:,.0f}")
print(f"Total Return: {total_return*100:.2f}%")
print(f"CAGR: {cagr*100:.2f}%")
print(f"Sharpe Ratio: {sharpe:.2f}")
print(f"Max Drawdown: {max_dd*100:.2f}%")
print(f"Win Rate: {win_rate:.2f}%")
print(f"Trades: {len(trades_df)}")
print(f"Trade Log Exported: {csv_path}")

# =========================
# PLOTLY CHART
# =========================
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Price"
))

# Buy markers
fig.add_trace(go.Scatter(
    x=df[df["signal"] == 1].index,
    y=df[df["signal"] == 1]["close"],
    mode="markers",
    marker=dict(symbol="triangle-up", size=10),
    name="Buy"
))

# Sell markers
fig.add_trace(go.Scatter(
    x=df[df["signal"] == -1].index,
    y=df[df["signal"] == -1]["close"],
    mode="markers",
    marker=dict(symbol="triangle-down", size=10),
    name="Sell"
))

# Remove gaps
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])

fig.update_layout(
    title="Modified Trend Bar Momentum Strategy",
    template="plotly_dark",
    height=750
)

fig.show()



========== PERFORMANCE ==========
Final Capital: ₹1,088,389
Total Return: 8.84%
CAGR: 2.10%
Sharpe Ratio: 0.30
Max Drawdown: -14.19%
Win Rate: 57.73%
Trades: 194
Trade Log Exported: modified_trend_bar_momentum_trades.csv


In [41]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from openalgo import api
from math import sqrt

# =========================
# CONFIG
# =========================
SYMBOL = "IRCON"
EXCHANGE = "NSE"
INTERVAL = "D"        # "D", "5m", "15m", etc.
START_DATE = "2026-01-01"
END_DATE = "2026-01-08"

INITIAL_CAPITAL = 1_000_000
RISK_PER_TRADE = 0.02   # 2%
STOP_LOSS_PCT = 0.01    # 1%
TAKE_PROFIT_PCT = 0.028 # 2.8%

# =========================
# CONNECT
# =========================
client = api(
    api_key="945cf5ba60f139afd48ce073fef0c346a2cc52d72f0c10c58e30acb50e0cb030",
    host="http://127.0.0.1:5000"
)

# =========================
# FETCH DATA
# =========================
df = client.history(
    symbol=SYMBOL,
    exchange=EXCHANGE,
    interval=INTERVAL,
    start_date=START_DATE,
    end_date=END_DATE
)

df.index = pd.to_datetime(df.index)

# =========================
# MODIFIED TREND BAR MOMENTUM STRATEGY
# =========================
df["prev_high"] = df["high"].shift(1)
df["prev_low"] = df["low"].shift(1)

df["signal"] = 0
df.loc[df["close"] > df["prev_high"], "signal"] = 1     # Buy
df.loc[df["close"] < df["prev_low"], "signal"] = -1     # Short

# =========================
# BACKTEST ENGINE
# =========================
capital = INITIAL_CAPITAL
position = 0
entry_price = 0
trade_log = []

equity_curve = []

for i in range(1, len(df)):
    row = df.iloc[i]
    price = row["close"]
    time = df.index[i]

    equity_curve.append(capital + position * (price - entry_price))

    # Exit logic
    if position != 0:
        if position == 1:
            sl = entry_price * (1 - STOP_LOSS_PCT)
            tp = entry_price * (1 + TAKE_PROFIT_PCT)
            exit_cond = price <= sl or price >= tp or row["signal"] == -1
        else:
            sl = entry_price * (1 + STOP_LOSS_PCT)
            tp = entry_price * (1 - TAKE_PROFIT_PCT)
            exit_cond = price >= sl or price <= tp or row["signal"] == 1

        if exit_cond:
            pnl = position * (price - entry_price)
            capital += pnl

            trade_log.append([entry_time, time, position, entry_price, price, pnl])

            position = 0

    # Entry logic
    if position == 0:
        risk_amount = capital * RISK_PER_TRADE
        qty = risk_amount / (price * STOP_LOSS_PCT)

        if row["signal"] == 1:
            position = qty
            entry_price = price
            entry_time = time

        elif row["signal"] == -1:
            position = -qty
            entry_price = price
            entry_time = time

# =========================
# TRADE LOG CSV
# =========================
trades_df = pd.DataFrame(trade_log, columns=[
    "Entry Time", "Exit Time", "Position", "Entry Price", "Exit Price", "PnL"
])

csv_path = "bar_up_down_trades.csv"
trades_df.to_csv(csv_path, index=False)

# =========================
# PERFORMANCE METRICS
# =========================
equity_series = pd.Series(equity_curve).dropna()
returns = equity_series.pct_change().dropna()

total_return = (capital - INITIAL_CAPITAL) / INITIAL_CAPITAL
years = len(df) / 252
cagr = (capital / INITIAL_CAPITAL) ** (1 / years) - 1 if years > 0 else 0

sharpe = sqrt(252) * returns.mean() / returns.std() if returns.std() != 0 else 0

drawdown = equity_series / equity_series.cummax() - 1
max_dd = drawdown.min()

win_rate = (trades_df["PnL"] > 0).mean() * 100 if len(trades_df) else 0

print("\n========== PERFORMANCE ==========")
print(f"Final Capital: ₹{capital:,.0f}")
print(f"Total Return: {total_return*100:.2f}%")
print(f"CAGR: {cagr*100:.2f}%")
print(f"Sharpe Ratio: {sharpe:.2f}")
print(f"Max Drawdown: {max_dd*100:.2f}%")
print(f"Win Rate: {win_rate:.2f}%")
print(f"Trades: {len(trades_df)}")
print(f"Trade Log Exported: {csv_path}")

# =========================
# PLOTLY CHART
# =========================
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Price"
))

# Buy markers
fig.add_trace(go.Scatter(
    x=df[df["signal"] == 1].index,
    y=df[df["signal"] == 1]["close"],
    mode="markers",
    marker=dict(symbol="triangle-up", size=10),
    name="Buy"
))

# Sell markers
fig.add_trace(go.Scatter(
    x=df[df["signal"] == -1].index,
    y=df[df["signal"] == -1]["close"],
    mode="markers",
    marker=dict(symbol="triangle-down", size=10),
    name="Sell"
))

# Remove gaps
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])

fig.update_layout(
    title="Modified Trend Bar Momentum Strategy",
    template="plotly_dark",
    height=750
)

fig.show()



========== PERFORMANCE ==========
Final Capital: ₹1,081,799
Total Return: 8.18%
CAGR: 2617.38%
Sharpe Ratio: 7.39
Max Drawdown: -0.45%
Win Rate: 100.00%
Trades: 1
Trade Log Exported: bar_up_down_trades.csv
